In [1]:
import requests
from bs4 import BeautifulSoup

In [2]:
def format_input(input_str):
    formatted_str = input_str.replace('\uFEFF', '').replace('\u00A0', ' ').replace("&nbsp;", " ").replace(" ", "-").replace("’", "").lower()
    return f"{formatted_str}/"

format_input("face to face")

In [4]:
url1 = 'https://www.theidioms.com/'

In [5]:
def do(x,c):
    for i in c:
        idiom = i.h5.text 
        if "zoom" in idiom:
            break
        current_list = [idiom]
        print(idiom)

        response = requests.get(url1 + format_input(idiom))
        soup = BeautifulSoup(response.text, 'html.parser')

        c = soup.find_all('ol')

        if c[1]=='<ol class="mpi"><li title="61 Thoughts on the devil is beating his wife"><a href="https://www.theidioms.com/the-devil-is-beating-his-wife/">the devil is beating his wife</a> <span class="icn">(61)</span></li><li title="19 Thoughts on raining cats and dogs"><a href="https://www.theidioms.com/rain-cats-and-dogs/">raining cats and dogs</a> <span class="icn">(19)</span></li><li title="17 Thoughts on break a leg"><a href="https://www.theidioms.com/break-a-leg/">break a leg</a> <span class="icn">(17)</span></li><li title="15 Thoughts on catch-22"><a href="https://www.theidioms.com/catch-22/">catch-22</a> <span class="icn">(15)</span></li><li title="12 Thoughts on apple of discord"><a href="https://www.theidioms.com/apple-of-discord/">apple of discord</a> <span class="icn">(12)</span></li><li title="12 Thoughts on a bed of roses"><a href="https://www.theidioms.com/bed-of-roses/">a bed of roses</a> <span class="icn">(12)</span></li><li title="11 Thoughts on home is where the heart is"><a href="https://www.theidioms.com/home-is-where-the-heart-is/">home is where the heart is</a> <span class="icn">(11)</span></li></ol>':
            c = soup.find_all('ul')

            for z in c[2]:
                current_list.append(z.text.replace('\xa0', ' '))

        else:
            for z in c[1]:
                current_list.append(z.text.replace('\xa0', ' '))

        x.append(current_list)


In [6]:
url = 'https://www.theidioms.com/list/'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

c=soup.find_all('article',class_='idiom')

In [7]:
x = []
do(x,c)

poke the bear
beauty is only skin deep
rub off
blow off steam
a shot in the dark
throw in the towel
let the cat out of the bag
fish rots from the head down
grit one’s teeth
drop dead


In [8]:
x[4]

['a shot in the dark',
 'It’s hard to tell the time without a clock, but I’ll take a shot in the dark and say it’s past midnight.',
 'I don’t have the map, but I’ll take a shot in the dark and try to find the restaurant on my own.',
 'Submitting my story to the competition is a bit of a shot in the dark, but who knows? Maybe they’ll like it.',
 'Driving on that narrow road in the fog felt like taking a shot in the dark; I could barely see anything.',
 'I can’t remember where I left my glasses, so I’ll take a shot in the dark and check the kitchen.']

In [9]:
url = 'https://www.theidioms.com/list/page/'

In [ ]:
for i in range(2,166):     # 166
    print(i)
    response = requests.get(url+str(i)+'/')
    soup = BeautifulSoup(response.text, 'html.parser')
    c=soup.find_all('article',class_='idiom')
    do(x,c)

In [42]:
y = [item for item in x if item[1] != 'the devil is beating his wife (61)' and item[1]!='the devil is beating his wife (62)']

In [46]:
for i in range(len(y)):
    for j in range(len(y[i])):
        y[i][j] = y[i][j].replace('\n', ' ')

In [50]:
with open('sentences_new.txt', 'w', encoding='utf-8') as sentences_file:
    for sublist in y:
        sentences_file.write('\n'.join(sublist[1:]) + '\n')

In [52]:
y[73]

['bottom line',
 'When the profit comes down, the bottom line is that this investment isn’t worth the risk.',
 'The bottom line is that we need to cut costs in order to remain competitive.',
 'At the end of everything, the bottom line is that I just want to be happy with my decision.',
 'We can debate it all we want, but the bottom line is that this is what needs to be done.',
 'The bottom line is that time is running out, and we need to act now.',
 'The bottom line is that pregnant women who smoke run a higher risk of harming their health.']

In [ ]:
for i in y:
    print(('\n'.join(i[1:])))

In [54]:
y[100]

['carry out',
 'The American scientist wanted to carry out several experiments before announcing the sending of humans to Mars.',
 'The boss carried out his orders.',
 'She carried out her logic flawlessly.',
 'I carried out the project according to schedule.',
 'We must carry out our duties promptly.',
 'Emma carried out her promise to Noah.']

Different functions to perform annotation

In [8]:
import spacy
nlp = spacy.load("en_core_web_md")

In [ ]:
def lemmatize(word):
    return nlp(word)[0].lemma_

def tag_consecutive_lemmatized(sentence, phrase):
    sentence_words = [lemmatize(word.text) for word in nlp(sentence)]
    phrase_words = [lemmatize(word.text) for word in nlp(phrase)]

    tagged_list = [0] * len(sentence_words)
    
    for i in range(len(sentence_words) - len(phrase_words) + 1):
        if sentence_words[i:i+len(phrase_words)] == phrase_words:
            for j in range(i, i+len(phrase_words)):
                tagged_list[j] = 1

    return tagged_list

# Example usage:
sentence_input = ""
phrase_input = ""

result = tag_consecutive_lemmatized(sentence_input, phrase_input)
print(result)

In [18]:
file_path = "the_idioms_sentences.txt"

with open(file_path, 'w', encoding = 'utf-8') as file:
    for row in y:
        file.write(row[1] + '\n')

More possible functions to tag the idioms

In [3]:
from difflib import SequenceMatcher

In [ ]:
def similar_phrase(sentence, phrase_to_label):
    doc = nlp(sentence)
    phrase_tokens = nlp(phrase_to_label)

    phrase_lemmas = [token.lemma_ for token in phrase_tokens]
    sentence_lemmas = [token.lemma_ for token in doc]

    result = [0] * len(doc)
    phrase_length = len(phrase_lemmas)
    for i in range(len(doc) - phrase_length + 1):
        current_sequence = sentence_lemmas[i:i + phrase_length]
        similarity = SequenceMatcher(None, phrase_lemmas, current_sequence).ratio()
        if similarity > 0.90:  # threshold
            result[i:i + phrase_length] = [1] * phrase_length

    return result

sentence = "He took a fresh cigarette offered by somebody else, lit it and then blew the smoke in Harry's face."
phrase = "blow smoke"

result_list = similar_phrase(sentence, phrase)
print(result_list)

In [ ]:
def label_phrase(sentence, phrase_to_label):
    doc = nlp(sentence)
    phrase_tokens = nlp(phrase_to_label)
    result = [0] * len(doc)
    phrase_length = len(phrase_tokens)
    for i in range(len(doc) - phrase_length + 1):
        if all(doc[i + j].text == phrase_tokens[j].text for j in range(phrase_length)):
            result[i:i + phrase_length] = [1] * phrase_length

    return result

sentence = "He took a fresh cigarette offered by somebody else, lit it and then blew the smoke in Harry's face."
phrase = "blow smoke"

result_list = label_phrase(sentence, phrase)
print(result_list)

In [ ]:
def label_phrase_static(sentence, phrase_to_label):

    doc = nlp(sentence)
    phrase_tokens = nlp(phrase_to_label)

    result = [0] * len(doc)
    phrase_length = len(phrase_tokens)
    for i in range(len(doc) - phrase_length + 1):
        if [token.text for token in doc[i:i + phrase_length]] == [token.text for token in phrase_tokens]:
            result[i:i + phrase_length] = [1] * phrase_length

    return result

sentence = "The general kept his hair on during the meeting this time."
phrase = "keep his hair on"

result_list = label_phrase_static(sentence, phrase)
print(result_list)